# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

T. Müller  ->  T. Müller  |  ['T. Müller']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
J. Li  ->  J. Li  |  ['J. Li']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Shi  ->  J. Shi  |  ['J. Shi']
J. Li  ->  J. Li  |  ['J. Li']


Arxiv has 59 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2504.17843
extracting tarball to tmp_2504.17843...

 done.


/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure Figures/Dustribution_3D
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure Figures/Dustribution_3D as tmp_2504.17843/./Figures/Dustribution_3D.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))


/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure Figures/I_tot
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure Figures/I_tot as tmp_2504.17843/./Figures/I_tot.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))


...

 done.


Found 196 bibliographic references in tmp_2504.17867/SMF.bbl.
Issues with the citations
syntax error in line 25: '=' expected
Retrieving document from  https://arxiv.org/e-print/2504.17877


extracting tarball to tmp_2504.17877... done.
Retrieving document from  https://arxiv.org/e-print/2504.18115
extracting tarball to tmp_2504.18115...

 done.
Retrieving document from  https://arxiv.org/e-print/2504.18202


extracting tarball to tmp_2504.18202... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.17843-b31b1b.svg)](https://arxiv.org/abs/2504.17843) | **A Nearby Dark Molecular Cloud in the Local Bubble Revealed via H$_2$ Fluorescence**  |
|| B. Burkhart, et al. -- incl., <mark>T. Müller</mark> |
|*Appeared on*| *2025-04-28*|
|*Comments*| *Accepted for publication in Nature Astronomy. Video of the Eos cloud: this http URL Interactive view of the Eos cloud and its relationship to the Sun and Local bubble: this http URL*|
|**Abstract**|            A longstanding prediction in interstellar theory posits that significant quantities of molecular gas, crucial for star formation, may be undetected due to being ``dark" in commonly used molecular gas tracers, such as carbon monoxide. We report the discovery of Eos, the closest dark molecular cloud, located just 94 parsecs from the Sun. This cloud is the first molecular cloud ever to be identified using H$_2$ far ultra-violet (FUV) fluorescent line emission, which traces molecular gas at the boundary layers of star-forming and supernova remnant regions. The cloud edge is outlined along the high-latitude side of the North Polar Spur, a prominent x-ray/radio structure. Our distance estimate utilizes 3D dust maps, the absorption of the soft X-ray background, and hot gas tracers such as O\,{\sc vi}; these place the cloud at a distance consistent with the Local Bubble's surface. Using high-latitude CO maps we note a small amount (M$_{\rm{H}_2}\approx$20-40\,M$_\odot$) of CO-bright cold molecular gas, in contrast with the much larger estimate of the cloud's true molecular mass (M$_{\rm{H}_2}\approx3.4\times 10^3$\,M$_\odot$), indicating most of the cloud is CO-dark. Combining observational data with novel analytical models and simulations, we predict this cloud will photoevaporate in 5.7 million years, placing key constraints on the role of stellar feedback in shaping the closest star-forming regions to the Sun.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.17867-b31b1b.svg)](https://arxiv.org/abs/2504.17867) | **Euclid preparation: TBD. Cosmic Dawn Survey: evolution of the galaxy stellar mass function across 0.2<z<6.5 measured over 10 square degrees**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-04-28*|
|*Comments*| *- Submitted to A&A - Catalogues available here: this https URL*|
|**Abstract**|            The Cosmic Dawn Survey Pre-launch (PL) catalogues cover an effective 10.13 deg$^{2}$ area with uniform deep Spitzer/IRAC data ($m\sim25$ mag, 5$\sigma$), the largest area covered to these depths in the infrared. These data are used to gain new insight into the growth of stellar mass across cosmic history by characterising the evolution of the galaxy stellar mass function (GSMF) through $0.2 < z \leq 6.5$. The total volume (0.62 Gpc$^{3}$) represents a tenfold increase compared to previous works that have explored $z > 3$ and significantly reduces cosmic variance, yielding strong constraints on the abundance of massive galaxies. Results are generally consistent with the literature but now provide firm estimates of number density where only upper limits were previously available. Contrasting the GSMF with the dark matter halo mass function suggests that massive galaxies ($M \gtrsim10^{11}$ M$_{\odot}$) at $z > 3.5$ required integrated star-formation efficiencies of $M/(M_{\rm h}f_{\rm b}) \gtrsim$ 0.25--0.5, in excess of the commonly-held view of ``universal peak efficiency" from studies on the stellar-to-halo mass relation (SHMR). Such increased efficiencies imply an evolving peak in the SHMR at $z > 3.5$ which can be maintained if feedback mechanisms from active galactic nuclei and stellar processes are ineffective at early times. In addition, a significant fraction of the most massive quiescent galaxies are observed to be in place already by $z\sim 2.5$--3. The apparent lack in change of their number density by $z\sim 0.2$ is consistent with relatively little mass growth from mergers. Utilising the unique volume, evidence for an environmental dependence of the galaxy stellar mass function is found all the way through $z\sim 3.5$ for the first time, though a more careful characterisation of the density field is ultimately required for confirmation.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.17877-b31b1b.svg)](https://arxiv.org/abs/2504.17877) | **The ALMA-CRISTAL survey: weak evidence for star-formation driven outflows in $z\sim5$ main-sequence galaxies**  |
|| J. E. Birkin, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-04-28*|
|*Comments*| *20 pages, 12 figures, accepted for publication in ApJ*|
|**Abstract**|            There is a broad consensus from theory that stellar feedback in galaxies at high redshifts is essential to their evolution, alongside conflicting evidence in the observational literature about its prevalence and efficacy. To this end, we utilize deep, high-resolution [CII] emission line data taken as part of the [CII] resolved ISM in star-forming galaxies with ALMA (CRISTAL) survey. Excluding sources with kinematic evidence for gravitational interactions, we perform a rigorous stacking analysis of the remaining 15 galaxies to search for broad emission features that are too weak to detect in the individual spectra, finding only weak evidence that a broad component is needed to explain the composite spectrum. Additionally, such evidence is mostly driven by CRISTAL-02, which is already known to exhibit strong outflows in multiple ISM phases. Interpreting modest residuals in the stack at $v\sim300$kms$^{-1}$ as an outflow, we derive a mass outflow rate of $\dot{M}_{\rm out}=26\pm11$M$_\odot$yr$^{-1}$ and a cold outflow mass-loading factor of $\eta_m=0.49\pm0.20$. This result holds for the subsample with the highest star-formation rate surface density $(\Sigma_{\rm{SFR}}>1.93$M$_\odot$yr$^{-1}$kpc$^{-2}$) but no such broad component is present in the composite of the lower-star-formation rate density subsample. Our results imply that the process of star-formation-driven feedback may already be in place in typical galaxies at $z=5$, but on average not strong enough to completely quench ongoing star formation.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.18115-b31b1b.svg)](https://arxiv.org/abs/2504.18115) | **Transverse Oscillations of Coronal Loops Induced by a Jet-Related Confined Flare on 11 July 2022**  |
|| M. Lin, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-04-28*|
|*Comments*| *22 pages, 12 figures, accepted for publication in Solar Physics*|
|**Abstract**|            In this article, we report the multiwavelength and multiview observations of transverse oscillations of two loop strands induced by a jet-related, confined flare in active region NOAA 13056 on 11 July 2022. The jet originates close to the right footpoint of the loops and propagates in the northeast direction. The average rise time and fall time of the jet are $\approx$ 11 and $\approx$ 13.5 minutes, so that the lifetime of the jet reaches $\approx$ 24.5 minutes. The rising motion of the jet is divided into two phases with average velocities of $\approx$ 164 and $\approx$ 546\,km\,s$^{-1}$. The falling motion of the jet is coherent with an average velocity of $\approx$ 124\,km\,s$^{-1}$. The transverse oscillations of the loops, lasting for 3 $-$ 5 cycles, are of fundamental standing kink mode. The maximal initial amplitudes of the two strands are $\approx$ 5.8 and $\approx$ 4.9 Mm. The average periods are $\approx$ 405\,s and $\approx$ 407\,s. Both of the strands experience slow expansions during oscillations. The lower limits of the kink speed are 895$_{-17}^{+21}$\,km\,s$^{-1}$ for loop\_1 and 891$_{-35}^{+29}$\,km\,s$^{-1}$ for loop\_2, respectively. The corresponding lower limits of the Alfvén speed are estimated to be 664$_{-13}^{+16}$\,km\,s$^{-1}$ and 661$_{-26}^{+22}$\,km\,s$^{-1}$.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.18202-b31b1b.svg)](https://arxiv.org/abs/2504.18202) | **A post-common-envelope binary with double-peaked Balmer emission lines from TMTS**  |
|| Q. Liu, et al. -- incl., <mark>X. Zhang</mark>, <mark>J. Shi</mark>, <mark>J. Li</mark> |
|*Appeared on*| *2025-04-28*|
|*Comments*| *8 pages, 4 figures, accepted for publication in A&A*|
|**Abstract**|            The dynamical method provides an efficient way to discover post-common-envelope binaries (PCEB) with faint white dwarfs (WDs), thanks to the development of time-domain survey projects. We perform a comprehensive analysis of the PCEB system TMTS J15530469+4457458 (J1553), discovered by the Tsinghua University-Ma Huateng Telescopes for Survey, to explore its physical origin and evolutionary fate. This system is characterized by double-peaked Balmer emission lines, and a cross-correlation function is applied to derive its radial velocity (RV) from a series of phase-resolved Keck spectra. Analyses with the cross-correlation function suggest that this system is a single-lined spectroscopic binary and only one star is optically visible. Further analysis through Doppler tomography indicates that J1553 is a detached binary without an accretion disk. Under such a configuration, the simultaneous light-curve and RV fitting reveal that this system contains an unseen WD with mass $M_{\rm A}=0.56\pm 0.09\, M_{\odot}$, and an M4 dwarf with mass $M_{\rm B}=0.37\pm 0.02\,M_{\odot}$ and radius $R_{\rm B}=0.403^{+0.014}_{-0.015}\,R_{\odot}$. The extra prominent Balmer emission lines seen in the spectra can trace the motion of the WD, which are likely formed near the WD surface as a result of wind accretion. According to the MESA simulation, J1553 could have evolved from a binary consisting of a 2.0-4.0 ${M}_{\odot}$ zero-age-main-sequence star and an M dwarf with an initial orbital period $P_i\approx 201-476$ d, and the system has undergone a common-envelope (CE) phase. After about $3.3\times10^6$ yr, J1553 should evolve into a cataclysmic variable, with a transient state as a supersoft X-ray source at the beginning. J1553 is an excellent system for studying wind accretion, CE ejection physics, and binary evolution theory.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2504.17843/./Figures/Extinction_Planck_GalfaHI_Xray_CO_Arrows.png', 'tmp_2504.17843/./Figures/Dustribution_3D.png', 'tmp_2504.17843/./Figures/Eos_UV_Smoothing.png']
copying  tmp_2504.17843/./Figures/Extinction_Planck_GalfaHI_Xray_CO_Arrows.png to _build/html/
copying  tmp_2504.17843/./Figures/Dustribution_3D.png to _build/html/
copying  tmp_2504.17843/./Figures/Eos_UV_Smoothing.png to _build/html/
exported in  _build/html/2504.17843.md
    + _build/html/tmp_2504.17843/./Figures/Extinction_Planck_GalfaHI_Xray_CO_Arrows.png
    + _build/html/tmp_2504.17843/./Figures/Dustribution_3D.png
    + _build/html/tmp_2504.17843/./Figures/Eos_UV_Smoothing.png
found figures ['tmp_2504.17867/./figures/smf_litcomp_0.2_3.5.png', 'tmp_2504.17867/./figures/smf_litcomp_3.5_6.5.png', 'tmp_2504.17867/./figures/tot_mcmc_examp.png']
copying  tmp_2504.17867/./figures/smf_litcomp_0.2_3.5.png to _build/html/
copying  tmp_2504.17867/./figures/smf_litcomp_3.5_6.5.png to _build/html/
copying  tm

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\actaa}{Acta Astron.}$
$\newcommand{\araa}{Annu. Rev. Astron. Astrophys.}$
$\newcommand{\areps}{Annu. Rev. Earth Planet. Sci.}$
$\newcommand{\aar}{Astron. Astrophys. Rev.}$
$\newcommand{\ab}{Astrobiology}$
$\newcommand{\aj}{Astron. J.}$
$\newcommand{\ac}{Astron. Comput.}$
$\newcommand{\apart}{Astropart. Phys.}$
$\newcommand{\apj}{Astrophys. J.}$
$\newcommand{\apjl}{Astrophys. J. Lett.}$
$\newcommand{\apjs}{Astrophys. J. Suppl. Ser.}$
$\newcommand{\ao}{Appl. Opt.}$
$\newcommand{\apss}{Astrophys. Space Sci.}$
$\newcommand{\aap}{Astron. Astrophys.}$
$\newcommand{\aapr}{Astron. Astrophys. Rev.}$
$\newcommand{\aaps}{Astron. Astrophys. Suppl.}$
$\newcommand{\baas}{Bull. Am. Astron. Soc.}$
$\newcommand{\bain}{Bulletin Astronomical Institute of the Netherlands}$
$\newcommand{\caa}{Chin. Astron. Astrophys.}$
$\newcommand{\cjaa}{Chin. J. Astron. Astrophys.}$
$\newcommand{\cqg}{Class. Quantum Gravity}$
$\newcommand{\epsl}{Earth Planet. Sci. Lett.}$
$\newcommand{\expa}{Exp. Astron.}$
$\newcommand{\frass}{Front. Astron. Space Sci.}$
$\newcommand{\gal}{Galaxies}$
$\newcommand{\gca}{Geochim. Cosmochim. Acta}$
$\newcommand{\grl}{Geophys. Res. Lett.}$
$\newcommand{\icarus}{Icarus}$
$\newcommand{\ija}{Int. J. Astrobiol.}$
$\newcommand{\jatis}{J. Astron. Telesc. Instrum. Syst.}$
$\newcommand{\jcap}{J. Cosmol. Astropart. Phys.}$
$\newcommand{\jgr}{J. Geophys. Res.}$
$\newcommand{\jgrp}{J. Geophys. Res.: Planets}$
$\newcommand{\jqsrt}{J. Quant. Spectrosc. Radiat. Transf.}$
$\newcommand{\lrca}{Living Rev. Comput. Astrophys.}$
$\newcommand{\lrr}{Living Rev. Relativ.}$
$\newcommand{\lrsp}{Living Rev. Sol. Phys.}$
$\newcommand{\memsai}{Mem. Soc. Astron. Italiana}$
$\newcommand{\maps}{Meteorit. Planet. Sci.}$
$\newcommand{\mnras}{Mon. Not. R. Astron. Soc.}$
$\newcommand{\nat}{Nature}$
$\newcommand{\nastro}{Nat. Astron.}$
$\newcommand{\ncomms}{Nat. Commun.}$
$\newcommand{\ngeo}{Nat. Geosci.}$
$\newcommand{\nphys}{Nat. Phys.}$
$\newcommand{\na}{New Astron.}$
$\newcommand{\nar}{New Astron. Rev.}$
$\newcommand{\physrep}{Phys. Rep.}$
$\newcommand{\pra}{Phys. Rev. A}$
$\newcommand{\prb}{Phys. Rev. B}$
$\newcommand{\prc}{Phys. Rev. C}$
$\newcommand{\prd}{Phys. Rev. D}$
$\newcommand{\pre}{Phys. Rev. E}$
$\newcommand{\prl}{Phys. Rev. Lett.}$
$\newcommand{\psj}{Planet. Sci. J.}$
$\newcommand{\planss}{Planet. Space Sci.}$
$\newcommand{\pnas}{Proc. Natl Acad. Sci. USA}$
$\newcommand{\procspie}{Proc. SPIE}$
$\newcommand{\pasa}{Publ. Astron. Soc. Aust.}$
$\newcommand{\pasj}{Publ. Astron. Soc. Jpn}$
$\newcommand{\pasp}{Publ. Astron. Soc. Pac.}$
$\newcommand{\raa}{Res. Astron. Astrophys.}$
$\newcommand{\rmxaa}{Rev. Mexicana Astron. Astrofis.}$
$\newcommand{\sci}{Science}$
$\newcommand{\sciadv}{Sci. Adv.}$
$\newcommand{\solphys}{Sol. Phys.}$
$\newcommand{\sovast}{Soviet Astron.}$
$\newcommand{\ssr}{Space Sci. Rev.}$
$\newcommand{\uni}{Universe}$
$\newcommand{\overbar}[1]{\mkern 1.5mu\overline{\mkern-1.5mu#1\mkern-1.5mu}\mkern 1.5mu}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\thickfrac}[2]{\genfrac{1.3pt}{#1}{#2}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\obs}{\ensuremath{\mathrm{obs}}}$
$\newcommand{\ini}{\ensuremath{\mathrm{ini}}}$
$\newcommand$
$\newcommand{\no}{\ifmmode{N_{\rm HI}} \else N_{\rm HI} \fi}$
$\newcommand{\nt}{\ifmmode{N_{\rm H_2}} \else N_{\rm HI} \fi}$
$\newcommand{\msun}{\ifmmode{\rm M_{\odot}}\else \rm M_{\odot}\fi}$
$\newcommand{\mpc}{\ifmmode{\rm M_{\odot}  pc^{-2}} \else \rm M_{\odot}  pc^{-2} \fi}$
$\newcommand{\tra}{\ifmmode  \text{HI-to-H}_2\else H{\small I}-to-H_2 \fi}$
$\newcommand{\aG}{\ifmmode{\alpha G}\else \alpha G \fi}$
$\newcommand{\iuv}{\ifmmode{I_{\rm UV}}\else I_{\rm UV} \fi}$
$\newcommand{\sg}{\ifmmode \sigma_{g} \else \sigma_{g} \fi}$</div>



<div id="title">

# A Nearby Dark Molecular Cloud in the Local Bubble Revealed via $H_2$ Fluorescence

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.17843-b31b1b.svg)](https://arxiv.org/abs/2504.17843)<mark>Appeared on: 2025-04-28</mark> -  _Accepted for publication in Nature Astronomy. Video of the Eos cloud: this http URL Interactive view of the Eos cloud and its relationship to the Sun and Local bubble: this http URL_

</div>
<div id="authors">

B. Burkhart, et al. -- incl., <mark>T. Müller</mark>

</div>
<div id="abstract">

**Abstract:** A longstanding prediction in interstellar theory posits that significant quantities of molecular gas, crucial for star formation, may be undetected due to being "dark" in commonly used molecular gas tracers, such as carbon monoxide.We report the discovery of Eos, the closest dark molecular cloud, located just 94 parsecs from the Sun.This cloud is the first molecular cloud ever to be identified using $H_2$ far ultra-violet (FUV) fluorescent line emission, which traces molecular gas at the boundary layers of star-forming and supernova remnant regions.The cloud edge is outlined along the high-latitude side of the North Polar Spur, a prominent x-ray/radio structure.Our distance estimate utilizes 3D dust maps, the absorption of the soft X-ray background, and hot gas tracers such as O ${\sc vi}$ ; these place the cloud at a distance consistent with the Local Bubble's surface.Using high-latitude CO maps we note a small amount (M $_{\rm{H}_2}\approx$ 20-40 M $_\odot$ ) of CO-bright cold molecular gas, in contrast with the much larger estimate of the cloud's true molecular mass (M $_{\rm{H}_2}\approx3.4\times 10^3$ M $_\odot$ ), indicating most of the cloud is CO-dark. Combining observational data with novel analytical models and simulations, we predict this cloud will photoevaporate in 5.7 million years, placing key constraints on the role of stellar feedback in shaping the closest star-forming regions to the Sun.

</div>

<div id="div_fig1">

<img src="tmp_2504.17843/./Figures/Extinction_Planck_GalfaHI_Xray_CO_Arrows.png" alt="Fig1" width="100%"/>

**Figure 1. -** The Eos cloud seen in other tracers. The purple contours in all images represent the $H_2$ emission contour from the fluorescent emission from Fig.\ref{fig:$joH_2$map}. Top Left: Total extinction derived by integrating the {\sc Dustribution} density along the line of sight. Top Middle: Column density map of the Eos cloud derived from Planck 545 GHz data 2016planck  assuming $T=10$ K, $\kappa_\nu=4.73$ cm$^{2}$ g$^{-1}$ and a distance of 100 pc. Top Right: GALFA-H {\sc i} column density map 2011ApJS..194...20P. Bottom Left and Middle: ROSAT 0.25 keV and 1 keV maps from 1991Sci...252.1529S.
      Purple contours are overlaid showing the location of strong $H_2$ fluorescence from Fig.\ref{fig:$joH_2$map}. The Eos cloud shows a prominent outline absorbing the soft X-ray flux and creating a bright X-ray halo towards lower Galactic longitude. The interaction region provides a nearby example of a hot-cold gas interface. Bottom Right: CO data from 2022ApJS..262....5D; the small CO-bright region (known as MBM 40) within the on-sky cloud boundary is shown by the white arrow.  (*fig:3ddustmap*)

</div>
<div id="div_fig2">

<img src="tmp_2504.17843/./Figures/Dustribution_3D.png" alt="Fig2" width="100%"/>

**Figure 2. -** 3D dust density structure of the Eos cloud. Top: 2D slices of the cloud at different distances The color bar indicates both the total dust extinction density as well as the total mass density, assuming dust properties from 2003ApJ...598.1017D and a gas-to-dust mass ratio of 124. As in Figure \ref{fig:$joH_2$map}, the magenta contour shows the region with high $H_2$/FUV ratio. Bottom Left: {\sc Dustribution} density for $b=55^\circ$. The two red lines indicate the minimum and maximum extent of the magenta contour in Fig. \ref{fig:3ddustmap}, and the cyan and green dashed lines indicate the extent of the Local Bubble at this latitude (from 2020A&A...636A..17P and 2024arXiv240304961O respectively). Bottom Right: Projected view of the Eos cloud isosurface and surroundings relative to Local Bubble models. There is no indication of other clouds in the same direction. The 3D dust density of the Solar neighborhood highlighting the Eos cloud and the Local Bubble models can be viewed at links provided in the Data Availability section.  (*fig:dust3D*)

</div>
<div id="div_fig3">

<img src="tmp_2504.17843/./Figures/Eos_UV_Smoothing.png" alt="Fig3" width="100%"/>

**Figure 3. -** All-sky map of $H_2$ fluorescence with and without the adaptive smoothing method of jo2017ApJS..231...21J shown in the top and bottom panels, respectively. The $H_2$ fluorescent emission studied here is a robust feature of the data regardless of whether adaptive smoothing is utilized or not.  (*fig:smoothing*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.17843"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcid}[1]$
$\newcommand{\arraystretch}{1.4}$
$\newcommand{\photoz}{photo-\textit{z}}$
$\newcommand{\specz}{spec-\textit{z}}$
$\newcommand{\Msol}{{\rm M}_\odot}$
$\newcommand{\classic}{\textsc{Classic}}$
$\newcommand{\lephare}{\texttt{LePHARE}}$
$\newcommand{\eazy}{\texttt{EAZY}}$
$\newcommand{\tractor}{\texttt{The Tractor}}$
$\newcommand{\farmer}{\texttt{The Farmer}}$
$\newcommand{\sigmaz}{\sigma_{\Delta z / (1+z_{\rm s})}}$
$\newcommand{\zmin}{z_{\rm phot}^{\rm min}}$
$\newcommand{\zmax}{z_{\rm phot}^{\rm max}}$
$\newcommand{\SEP}{\texttt{SEP}}$
$\newcommand{\chOne}{[3.6\mum]}$
$\newcommand{\chTwo}{[4.5\mum]}$
$\newcommand{\hscpipe}{\texttt{hscpipe}}$
$\newcommand{\Ms}{\mathcal{M}\xspace}$</div>



<div id="title">

# $\Euclid$\/ preparation. Cosmic Dawn Survey:\\evolution of the galaxy stellar mass function across $0.2 < z \leq 6.5$ measured over 10 square degrees

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.17867-b31b1b.svg)](https://arxiv.org/abs/2504.17867)<mark>Appeared on: 2025-04-28</mark> -  _- Submitted to A&A - Catalogues available here: this https URL_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** The Cosmic Dawn Survey Pre-launch (PL) catalogues cover an effective 10.13 deg $^{2}$ area with uniform deep _Spitzer_ /IRAC data ( $m\sim25$ mag, 5 $\sigma$ ), the largest area covered to these depths at infrared wavelengths. These data are used to gain new insight into the growth of stellar mass across cosmic history by characterising the evolution of the galaxy stellar mass function through $0.2 < z \leq 6.5$ . The total volume (0.62 Gpc $^{3}$ ) represents an order of magnitude increase compared to previous works that have explored $z > 3$ and significantly reduces cosmic variance, thus yielding strong constraints on the abundance of galaxies above the characteristic stellar mass ( $\Ms$ $^{\star}$ ) across this ten billion year time period. The evolution of the galaxy stellar mass function is generally consistent with results from the literature but now provide firm estimates of number density where  only upper limits were previously available.Contrasting the galaxy stellar mass function with the dark matter halo mass function suggests that massive galaxies ( $\mathcal{M} \gtrsim10^{11}$ M $_{\odot}$ ) at $z > 3.5$ required integrated star-formation efficiencies of $\mathcal{M}/(\mathcal{M}_{\rm h}f_{\rm b}) \gtrsim$ 0.25--0.5, in excess of the commonly-held view of $\enquote{universal peak efficiency}$ from studies on the stellar-to-halo mass relation. Such increased efficiencies imply an evolving peak in the stellar-to-halo mass relation at $z > 3.5$ which can be maintained if feedback mechanisms from active galactic nuclei and stellar processes are ineffective at early times. In addition, a significant fraction of the most massive quiescent galaxies are observed to be in place already by $z\sim 2.5$ --3. The apparent lack in change of their number density by $z\sim 0.2$ is consistent with relatively little mass growth from mergers. Utilising the unique volume, evidence for an environmental dependence of the galaxy stellar mass function is found all the way through $z\sim 3.5$ for the first time, though a more careful characterisation of the density field is ultimately required for confirmation.

</div>

<div id="div_fig1">

<img src="tmp_2504.17867/./figures/smf_litcomp_0.2_3.5.png" alt="Fig11" width="100%"/>

**Figure 11. -** The observed total galaxy stellar mass function of DAWN PL (coloured points) and uncertainties (1$\sigma$, shaded) compared with two measurements from COSMOS, \citetalias{Davidzon2017} and \citetalias{Weaver2023SMF}, through to $3 < z \leq 3.5$. $\Ms$  bins below the stellar mass limit (Eq. \ref{eq:tot_mcomp}) are not shown. The volume limit according to the $N=0$ Poisson uncertainty is shown by the downward pointing coloured arrow for DAWN PL and in grey for \citetalias{Weaver2023SMF}. (*fig:smf_litcomp*)

</div>
<div id="div_fig2">

<img src="tmp_2504.17867/./figures/smf_litcomp_3.5_6.5.png" alt="Fig12" width="100%"/>

**Figure 12. -** Same as Fig. \ref{fig:smf_litcomp}, but at $z > 3.5$ and also showing measurements from \citetalias{Grazian2015} with HST (grey circles) and \citetalias{Weibel2024} with JWST (grey squares).  (*fig:smf_litcomp2*)

</div>
<div id="div_fig3">

<img src="tmp_2504.17867/./figures/tot_mcmc_examp.png" alt="Fig1" width="100%"/>

**Figure 1. -** Variation in the best-fit double Schechter function resulting from three different parameterisations of the uncertainty in $\Ms$  used for the Eddington bias kernel (Eq. \ref{eq:kernel}) at $1.1 < z \leq 1.5$. The shape of the kernel according to each parameterisation is shown in the inset plot, with the corresponding parameter values annotated above in matching colours. The resulting intrinsic (solid) and observed (dotted) galaxy stellar mass function is shown for each case, with the value of the best-fit characteristic mass ($\Ms$$^{\star}$) annotated and given by the vertical dashed lines. The treatment of the Eddington bias significantly affects the inferred intrinsic galaxy stellar mass function. (*fig:tot_mcmc_examp*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.17867"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

452  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
